### Initializing

In [2]:
#imports
import pandas as pd
import bisect
import numpy as np 
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
wordnet_lemma = WordNetLemmatizer()
from nltk.stem import SnowballStemmer 
english_stemmer = SnowballStemmer('english')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Yael\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [45]:
#Reading file and axamining general info
credit_score=pd.read_csv('/datasets/credit_scoring_eng.csv')
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


2174 missing values in float64 columns days_employed, total_income 

### Data preprocessing

#### Processing missing values

In [46]:
#all missing values in days_employed correspond to missing values in total_income
print(credit_score[credit_score['days_employed'].isnull()]['total_income'].unique())
print(credit_score['days_employed'].min())
print(credit_score['days_employed'].max())

#map each days_employed null to average days_employed for the correspondind age, the same for toatl_income

#print(credit_score[credit_score['days_employed']>=0]['days_employed'].min())
#print(credit_score[credit_score['total_income']>=0]['total_income'].min())
days_mapper_worker = credit_score[credit_score['days_employed']<=0].groupby('dob_years')['days_employed'].mean().to_dict()
#print(days_mapper_worker)
days_mapper_retiree = credit_score[credit_score['days_employed']>0].groupby('dob_years')['days_employed'].mean().to_dict()
#print(days_mapper_retiree)
income_mapper_worker = credit_score[credit_score['days_employed']<=0].groupby('dob_years')['total_income'].mean().to_dict()
income_mapper_retiree = credit_score[credit_score['days_employed']>0].groupby('dob_years')['total_income'].mean().to_dict()

def manage_nulls(row, column_name,mapper_worker, mapper_retiree):
    #map null cells to the values in the input dicts, non-nulls are unchanged
    if np.isnan(row[column_name]) or row[column_name] is None:
        if row['income_type'] == 'retiree':           
            return list(mapper_retiree.values())[bisect.bisect_left(list(mapper_retiree.keys()), row['dob_years'])]
        else:
            return list(mapper_worker.values())[bisect.bisect_left(list(mapper_worker.keys()), row['dob_years'])]
    else:
        return (row[column_name])

credit_score['days_employed']=credit_score.apply(manage_nulls, axis=1, args=('days_employed',days_mapper_worker,days_mapper_retiree))
credit_score['total_income']=credit_score.apply(manage_nulls, axis=1, args=('total_income',income_mapper_worker,income_mapper_retiree))
      
credit_score.info()

[nan]
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []
False
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


We have nulls in days_employed and the corresponding total_income rows.
Each null is mapped to the average days / income of his age (dob_years). 
since days_employed has extremely high values for retirees which affect the average, two separated dicts are used for retirees (days>0) and non-retirees (days<=0). 
if a customer with null has income_type == retiree the mapping is done using the retiree dict.

#### Data type replacement

In [47]:
credit_score=credit_score.astype({'days_employed' : 'int64', 'total_income' : 'int64'})
print(credit_score.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


#### Processing duplicates

In [48]:
#print(credit_score['education'].value_counts())

for column in credit_score.columns:
    if credit_score[column].dtype == 'object':
        credit_score[column]=credit_score[column].str.lower()
        
credit_score=credit_score.drop_duplicates().reset_index(drop=True)
print(credit_score.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


71 identical case insensitive duplicates removed

### Lemmatization

In [49]:
#print(credit_score['education'].value_counts())
#print(credit_score['family_status'].value_counts())
#print(credit_score['gender'].value_counts())
#print(credit_score['income_type'].value_counts())
print(credit_score['purpose'].value_counts())

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.tag.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

#create a list of all unique purpose variations and use them as keys of a mapping dict
purpose_keys=credit_score['purpose'].unique()
purpose_dict=dict.fromkeys(purpose_keys)
#print(purpose_keys)

#map each purpose key to a general category using lammatization and stemming of purpose column
for key in purpose_keys:
    tokens = nltk.word_tokenize(key)
    lemmas = [wordnet_lemma.lemmatize(w, get_wordnet_pos(w)) for w in tokens]
    stems = []
    for lemma in lemmas:
        stems.append(english_stemmer.stem(lemma))
    if english_stemmer.stem('wedding') in stems: 
        purpose_dict[key] = 'wedding'
    elif english_stemmer.stem('estate') in stems or english_stemmer.stem('house') in stems or english_stemmer.stem('property') in stems:
        purpose_dict[key] = 'real estate'
    elif english_stemmer.stem('car') in stems:
        purpose_dict[key] = 'car'
    elif english_stemmer.stem('university') in stems or english_stemmer.stem('education') in stems:
        purpose_dict[key] = 'education'    
    
#print(purpose_dict)

#create a new column in our dataFrame which maps purpose columns to the general categories according to the dictionary
credit_score['purpose_category'] = credit_score['purpose'].map(purpose_dict)
#now we can examine the true statistics of loan purposes 
print(credit_score['purpose_category'].value_counts())


wedding ceremony                                 791
having a wedding                                 768
to have a wedding                                765
real estate transactions                         675
buy commercial real estate                       661
housing transactions                             652
buying property for renting out                  651
transactions with the residential real estate    650
housing                                          646
purchase of the house                            646
purchase of the house for my family              638
construction of own property                     635
property                                         633
transactions with my real estate                 627
building a real estate                           624
buy real estate                                  621
purchase of my own house                         620
building a property                              619
property renovation                           

Customers take loans for same common purposes which can be united to 4 categories. 
~50% of loans were taken for real estate (residential or commercial)
~20% of loans were taken for purchasing a car or funding education
~10% of loans were taken for funding a wedding

### Categorizing Data

In [50]:
#print(credit_score['children'].value_counts())
#print(credit_score.loc[credit_score['days_employed']>0,['days_employed','total_income', 'income_type', 'debt','dob_years']].head(40))
#print(credit_score[credit_score['days_employed']>0]['days_employed'].value_counts())
#print(credit_score['dob_years'].value_counts())
#print(credit_score['education'].value_counts())
#print(credit_score['education_id'].value_counts())
#print(credit_score['family_status'].value_counts())
#print(credit_score['family_status_id'].value_counts())
#print(credit_score['gender'].value_counts())
#print(credit_score['income_type'].value_counts())
#print(credit_score['debt'].value_counts())
#print(credit_score[credit_score['total_income']!=0]['total_income'].min())

In [51]:
#categorizing children
def has_children(children):
    if children > 0:
        return 1
    else:
        return 0
credit_score['has_children']=credit_score['children'].apply(has_children)


In [52]:
#categorizing days_employed
def employment_years(days):
    days = -1*days
    if days < 0:
        return 'retiree'
    elif days == 0:
        return 'unemployed'
    elif days < 365:
        return '0-1'
    elif days < 5*365:
        return '2-5'
    elif days < 10*365:
        return '5-10'
    elif days < 20*365:
        return '10-20'
    elif days < 30*365:
        return '20-30'
    else:
        return '>30'
credit_score['employment_years']=credit_score['days_employed'].apply(employment_years)
print(credit_score['employment_years'].value_counts())



2-5        7260
5-10       5324
retiree    3831
10-20      2455
0-1        1827
20-30       579
>30         178
Name: employment_years, dtype: int64


From examining both days and income type we can see all positive values are reffered retirees. These values make no sence (correspond to 900 years...) so we will ignore them during the analysis and will only use the category.
The negative values of days reffer to employed and therefore categorized to years chunks

In [53]:
#categorizing dob_years
def age_group(age):
    if age < 18:
        return 'child'
    elif age < 35:
        return 'young-adult'
    elif age < 55:
        return 'middle-aged adult'
    elif age < 65:
        return 'older-adult'
    else:
        return 'senior'
credit_score['age_group']=credit_score['dob_years'].apply(age_group)
print(credit_score['age_group'].value_counts())


middle-aged adult    10607
young-adult           5967
older-adult           3884
senior                 895
child                  101
Name: age_group, dtype: int64


Age is categorized to 5 common age groups

In [54]:
#categorizing total_income
percentile_20 = np.percentile(credit_score['total_income'],20)
percentile_40 = np.percentile(credit_score['total_income'],40)
percentile_60 = np.percentile(credit_score['total_income'],60)
percentile_80 = np.percentile(credit_score['total_income'],80)

def total_income_percentile(income, percentile_20, percentile_40,percentile_60,percentile_80):
    if income < percentile_20:
        return '0-20'
    elif income < percentile_40:
        return '20-40'
    elif income < percentile_60:
        return '40-60'
    elif income < percentile_80:
        return '60-80'
    else: 
        return '80-100'
    
credit_score['total_income_percentile']=credit_score['total_income'].apply(total_income_percentile, args=(percentile_20, percentile_40,percentile_60,percentile_80))
print(credit_score['total_income_percentile'].value_counts())


60-80     4306
20-40     4291
0-20      4291
80-100    4291
40-60     4275
Name: total_income_percentile, dtype: int64


Total_income is categorized to 5 groups according to the percentile of each.


### Analysis

In [66]:
#Is there a relation between having children and being a debtor?

print(credit_score.groupby('has_children')['debt'].value_counts())
#calculating probability to have debt
(credit_score.groupby('has_children')['debt'].mean())

has_children  debt
0             0       13074
              1        1064
1             0        6639
              1         677
Name: debt, dtype: int64


has_children
0    0.075258
1    0.092537
Name: debt, dtype: float64

If a customer has children he has a probability of 0.9 to pay a loan on time.
If a customer has no children he has a probability of 0.92 to pay a loan on time.
There is almost no relation between having children and being a debtor, only a tiny advantage for customers with no children.

In [65]:
#Is there a relation between marital status and being a debtor?

print(credit_score.groupby('family_status')['debt'].value_counts())
#calculating probability to have debt
(pd.pivot_table(credit_score, values='debt', index=['family_status'], aggfunc=np.mean))


family_status      debt
civil partnership  0        3763
                   1         388
divorced           0        1110
                   1          85
married            0       11408
                   1         931
unmarried          0        2536
                   1         274
widow / widower    0         896
                   1          63
Name: debt, dtype: int64


,debt
family_status,
civil partnership,0.093471
divorced,0.071130
married,0.075452
unmarried,0.097509
widow / widower,0.065693


There is almost no correlation between marital status and being a debtor. 

In [64]:
#Is there a relation between income level and being a debtor

print(credit_score.groupby('total_income_percentile')['debt'].value_counts())
#calculating probability to have debt
(pd.pivot_table(credit_score, values='debt', index=['total_income_percentile'], aggfunc=np.mean))


total_income_percentile  debt
0-20                     0       3947
                         1        344
20-40                    0       3939
                         1        352
40-60                    0       3889
                         1        386
60-80                    0       3946
                         1        360
80-100                   0       3992
                         1        299
Name: debt, dtype: int64


,debt
total_income_percentile,
0-20,0.080168
20-40,0.082032
40-60,0.090292
60-80,0.083604
80-100,0.069681


there is almost no relation between income level and being a debtor

In [63]:
#How do different loan purposes affect on-time repayment of the loan?

print(credit_score.groupby('purpose_category')['debt'].value_counts())
#calculating probability to have debt
(pd.pivot_table(credit_score, values='debt', index=['purpose_category'], aggfunc=np.mean))


purpose_category  debt
car               0        3903
                  1         403
education         0        3643
                  1         370
real estate       0       10029
                  1         782
wedding           0        2138
                  1         186
Name: debt, dtype: int64


,debt
purpose_category,
car,0.093590
education,0.092200
real estate,0.072334
wedding,0.080034


Different loan purposes have similar probabilities for being a debtor. 

### Conclusion

The analysis above indicate that there are only minor differences in the probabilities to be a debtor when examining different parameters. Although for some categories we have more data (more customers) than others, when looking at each category independently we find that the probalility to pay a loan on time is around 0.9.